In [ ]:
import openai
import os
from dotenv import load_dotenv
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document as LDocument
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import pymysql
import locale
import gradio as gr

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

# MySQL 데이터베이스 연결 함수 (예시)
def connect_to_db():
    return pymysql.connect(
        host='database-1.cxcqeqcc6xxo.ap-northeast-2.rds.amazonaws.com',
        port=3306,
        user='admin',
        passwd='sesaclangchain',
        db='sesaclangchain',
        charset="utf8mb4"
    )

# 온라인 강의와 오프라인 강의 정보를 가져오는 함수
def fetch_courses():
    connection = connect_to_db()
    cursor = connection.cursor(pymysql.cursors.DictCursor)

    # 온라인 강의 정보 가져오기
    online_query = "SELECT * FROM online_course_tbl"
    cursor.execute(online_query)
    online_courses = cursor.fetchall()

    # 오프라인 강의 정보 가져오기
    offline_query = "SELECT * FROM offline_course_tbl"
    cursor.execute(offline_query)
    offline_courses = cursor.fetchall()

    connection.close()

    return online_courses, offline_courses

# 강의 정보를 문서 형식으로 변환
def convert_to_documents(online_courses, offline_courses):
    documents = []

    # 온라인 강의를 문서로 변환
    for course in online_courses:
        doc_text = f"강의 제목: {course['course_title']}\n강의 분류: {course['course_category']}\n강의 난도: {course['course_difficulty']}\n교육 목표: {course['course_objective']}\n수강 대상: {course['target_audience']}"
        documents.append(LDocument(page_content=doc_text))

    # 오프라인 강의를 문서로 변환
    for course in offline_courses:
        doc_text = f"강의 제목: {course['course_title']}\n강의 분류: {course['course_category']}\n교육 장소: {course['education_location']}\n교육 목표: {course['educational_goals']}\n수강 대상: {course['target_audience']}"
        documents.append(LDocument(page_content=doc_text))

    return documents

# 벡터 스토어 생성 (FAISS 사용)
def create_vector_store(documents):
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(documents, embeddings)
    return vector_store

# 질문과 관련된 강의를 검색하고 답변을 생성하는 챗봇
def create_chatbot(vector_store, user_prompt, system_prompt):
    # OpenAI LLM 설정
    llm = OpenAI(temperature=0.2)

    # 프롬프트 템플릿 생성
    prompt_template = """
    시스템 프롬프트: {system_message}
    사용자 프롬프트: {user_message}
    사용자의 질문: {question}
    """

    prompt = PromptTemplate(
        input_variables=["system_message", "user_message", "question"],
        template=prompt_template
    )

    # PromptTemplate을 사용하여 LLM을 설정
    chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

    # ConversationalRetrievalChain 생성
    retrieval_chain = ConversationalRetrievalChain.from_llm(
        llm=chat_model,
        retriever=vector_store.as_retriever(),
        return_source_documents=True
    )

    return retrieval_chain

# 사용자 질문에 대한 답변을 생성하는 함수
def get_answer(question, retrieval_chain, chat_history):
    result = retrieval_chain({"question": question, "chat_history": chat_history})

    answer = result.get('answer', '')
    source_documents = result.get('source_documents', None)

    return answer, source_documents

# main 함수
def main():
    # 시스템 로케일을 'ko_KR.UTF-8'로 설정하여 한글 깨짐을 방지
    
    locale.setlocale(locale.LC_ALL, 'ko_KR.UTF-8')

    # 강의 데이터 가져오기
    online_courses, offline_courses = fetch_courses()

    # 강의 정보를 문서로 변환
    documents = convert_to_documents(online_courses, offline_courses)

    # 벡터 스토어 생성
    vector_store = create_vector_store(documents)

    # 사용자 프롬프트와 시스템 프롬프트 정의
    system_prompt = "당신은 싹가능이 만든 한국의 도시 서울에서 제공하는 청년취업사관학교 챗봇 싹톡입니다. 당신의 임무는 주어진 데이터를 토대로 정보를 알려주세요."
    user_prompt = "당신에게 주어진 정보는 온라인/오프라인 강의 정보와 리뷰, 질문답하기 등 교육프로그램과 관련된 데이터를 가지고 있으며, 그 외 질문에는 답변이 어렵다고 말하세요."

    # 챗봇 생성
    retrieval_chain = create_chatbot(vector_store, user_prompt, system_prompt)

    # 대화 기록 초기화
    chat_history = []

    # Gradio 인터페이스 생성
    def chat_interface(user_input):
        # 답변 생성
        answer, _ = get_answer(user_input, retrieval_chain, chat_history)

        # 대화 기록에 (사용자 질문, 챗봇 답변) 추가
        chat_history.append((user_input, answer))

        # 챗봇의 응답을 (질문, 응답) 형태로 반환
        return [(user_input, answer)]

    with gr.Blocks() as demo:
        gr.Markdown("""
        <div style="text-align: center;">
            <img src="https://sesac.seoul.kr/static/common/images/www/common/logo.png" alt="logo" style="display: inline; width: 25px;"/>
            <h1 style="display:inline; color: #6FC274;">SESAC</h1>
        </div>
        """)
        chatbot = gr.Chatbot()
        user_input = gr.Textbox(label="질문을 입력하세요", placeholder="질문을 입력하세요", interactive=True )
        user_input.submit(chat_interface, inputs=user_input, outputs=chatbot)

    demo.launch()

# 프로그램 실행
if __name__ == "__main__":
    main()
